In [1]:
using Dates
using JuMP
using CSV
using DataFrames
using LinearAlgebra
using Pkg
using IJulia


In [2]:
ENV["COLUMNS"] = 200;

In [3]:
# Assuming your Julia file is named "COVIDResourceAllocation.jl"
include("COVIDResourceAllocation.jl")
using .COVIDResourceAllocation


In [4]:
states = ["CT", "DE", "MA", "MD", "ME", "NH", "NJ", "NY", "PA", "RI", "VT"]
start_date = Date(2020, 4, 1)
end_date   = Date(2020, 5, 1)
travel_threshold_hours = 4.0;

In [5]:
pct_nurses_available = 0.5
patients_per_nurse_covid = 2.5
nurse_hrs_per_week_covid = 36

@show nurse_hrs_per_day_covid = nurse_hrs_per_week_covid / 7
@show nurses_days_per_day_covid = 24 / nurse_hrs_per_day_covid

@show nurse_days_per_patient_day_covid = nurses_days_per_day_covid / patients_per_nurse_covid;

nurse_hrs_per_day_covid = nurse_hrs_per_week_covid / 7 = 5.142857142857143
nurses_days_per_day_covid = 24 / nurse_hrs_per_day_covid = 4.666666666666666
nurse_days_per_patient_day_covid = nurses_days_per_day_covid / patients_per_nurse_covid = 1.8666666666666665


In [6]:
adj = adjacencies(states, level=:state, source=:google, threshold=travel_threshold_hours);

In [7]:
forecast_active = forecast(
    states, start_date, end_date,
    level=:state,
    source=:ihme,
    forecast_type=:active,
    patient_type=:regular,
    bound_type=:mean,
);

In [8]:
demand = forecast_active * Float32(nurse_days_per_patient_day_covid);

In [9]:
nurses = n_nurses(states) * Float32(pct_nurses_available);

In [10]:
_nurses_beds = n_nurses(states, source=:beds);
_nurses_empl = n_nurses(states, source=:employment);
_nurses_ahrf = n_nurses(states, source=:ahrf);
DataFrame(
    state = states,
    from_beds = _nurses_beds,
    from_empl = _nurses_empl,
    from_ahrf = _nurses_ahrf,
)

Row,state,from_beds,from_empl,from_ahrf
,String,Float32,Float32,Float32
1,CT,8221.27,12120.0,4593.0
2,DE,2257.27,3234.06,1322.0
3,MA,17563.5,38220.5,10146.0
4,MD,9598.4,29581.0,6169.0
5,ME,2877.47,2346.56,1962.0
6,NH,2567.13,1484.33,1934.0
7,NJ,19164.1,71498.5,6652.0
8,NY,44213.9,83342.8,19547.0
9,PA,34385.9,65427.3,14189.0


In [11]:
model = reusable_resource_allocation(
    nurses,
    zeros(Float32, size(demand)...),
    demand,
    adj,
    send_new_only=false,
    sendrecieve_switch_time=0,
    min_send_amt=0,
    smoothness_penalty=0,
    setup_cost=0,
    sent_penalty=0.01,
    verbose=true
)
sent = value.(model[:sent])
println("termination status: ", termination_status(model))
println("solve time: ", round(solve_time(model), digits=3), "s")
println("objective function value: ", round(objective_value(model), digits=3))

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-21
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 22.6.0 22G513)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4492 rows, 4092 columns and 218023 nonzeros
Model fingerprint: 0x0a2522c1
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 2e+04]
Presolve removed 3812 rows and 1893 columns
Presolve time: 0.08s
Presolved: 680 rows, 2199 columns, 119365 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier performed 0 iterations in 0.14 seconds (0.06 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Iteration    Objective       Primal Inf.    Dual Inf.      Time
  

0.169s
objective function value: 

233461.741


In [12]:
results = NurseAllocationResults.results_all(sent, nurses, demand, states, start_date);

In [13]:
println("Total sent: ", results.total_sent)
println("Total shortage: ", results.total_shortage)
println("Average load: ", results.average_load)

Total sent: 28886.43657684326
Total shortage: 214557.67404556274
Average load: 1.1737886621449873


In [14]:
results.summary_table

Row,state,total_sent,total_received,initial_nurses,total_nurse_days,total_demand,total_shortage,average_load
,String,Float64,Float64,Float32,Float64,Float32,Float64,Float64
1,CT,1291.93,2416.66,2296.5,80094.3,96222.5,17242.7,1.24039
2,DE,555.537,233.057,661.0,5899.0,6673.62,874.505,1.11919
3,MA,4612.29,2154.04,5073.0,52174.5,64383.0,13474.5,1.22305
4,MD,1873.89,2005.66,3084.5,71409.5,79474.2,9775.27,1.1921
5,ME,932.525,0.0,981.0,1939.56,2261.99,359.406,1.1621
6,NH,887.499,27.0914,967.0,3653.0,4054.2,1256.99,1.34465
7,NJ,0.0,6029.5,3326.0,268491.0,3.05234e5,43564.7,1.12935
8,NY,11550.8,11335.4,9773.5,454958.0,5.72581e5,1.22314e5,1.27887
9,PA,5546.13,3505.87,7094.5,1.09274e5,108595.0,3482.5,0.991604


In [15]:
results.sent_matrix_table

Row,state,CT,DE,MA,MD,ME,NH,NJ,NY,PA,RI,VT
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,CT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1291.93,0.0,0.0,0.0
2,DE,0.0,0.0,0.0,0.0,0.0,0.0,555.537,0.0,0.0,0.0,0.0
3,MA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4612.29,0.0,0.0,0.0
4,MD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1873.89,0.0,0.0,0.0
5,ME,0.0,0.0,0.0,0.0,0.0,26.9531,0.0,0.0,0.0,905.571,0.0
6,NH,738.204,0.0,149.295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,NJ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,NY,1678.46,233.057,1701.27,2005.66,0.0,0.0,2152.95,0.0,3505.87,273.571,0.0
9,PA,0.0,0.0,0.0,0.0,0.0,0.0,3321.01,2225.12,0.0,0.0,0.0


In [16]:
println("First day:")
filter(row -> row.date == start_date, results.complete_table)

First day:


Row,state,date,sent,received,initial_nurses,current_nurses,demand,shortage,load,sent_to,sent_from
,String,Date,Float64,Float64,Float32,Float64,Float32,Float64,Float64,Any,Any
1,CT,2020-04-01,1291.93,344.713,2296.5,1349.28,1004.57,0.0,0.744521,"[(""NY"", 1291.93)]","[(""NH"", 344.713)]"
2,DE,2020-04-01,555.537,0.0,661.0,105.463,70.3982,0.0,0.667518,"[(""NJ"", 555.537)]",[]
3,MA,2020-04-01,4567.76,452.732,5073.0,957.972,505.24,0.0,0.527406,"[(""NY"", 4567.76)]","[(""NH"", 149.295), (""VT"", 303.437)]"
4,MD,2020-04-01,1873.89,0.0,3084.5,1210.61,643.135,0.0,0.531251,"[(""NY"", 1873.89)]",[]
5,ME,2020-04-01,905.571,0.0,981.0,75.4285,56.6743,0.0,0.751364,"[(""RI"", 905.571)]",[]
6,NH,2020-04-01,494.008,0.0,967.0,472.992,52.7495,0.0,0.111523,"[(""CT"", 344.713), (""MA"", 149.295)]",[]
7,NJ,2020-04-01,0.0,3876.55,3326.0,7202.55,4499.34,0.0,0.624687,[],"[(""DE"", 555.537), (""PA"", 3321.01)]"
8,NY,2020-04-01,0.0,8904.79,9773.5,18678.3,16920.3,0.0,0.905879,[],"[(""CT"", 1291.93), (""MA"", 4567.76), (""MD"", 1873.89), (""PA"", 659.073), (""RI"", 512.137)]"
9,PA,2020-04-01,3980.09,0.0,7094.5,3114.41,883.269,0.0,0.283607,"[(""NJ"", 3321.01), (""NY"", 659.073)]",[]


In [17]:
s = "NY"
filter(row -> row.state == s, results.complete_table)

Row,state,date,sent,received,initial_nurses,current_nurses,demand,shortage,load,sent_to,sent_from
,String,Date,Float64,Float64,Float32,Float64,Float32,Float64,Float64,Any,Any
1,NY,2020-04-01,0.0,8904.79,9773.5,18678.3,16920.3,0.0,0.905879,[],"[(""CT"", 1291.93), (""MA"", 4567.76), (""MD"", 1873.89), (""PA"", 659.073), (""RI"", 512.137)]"
2,NY,2020-04-02,0.0,1644.95,9773.5,20323.2,18678.3,0.0,0.919061,[],"[(""MA"", 44.5293), (""PA"", 1566.05), (""RI"", 34.3784)]"
3,NY,2020-04-03,0.0,785.664,9773.5,21108.9,20323.2,0.0,0.96278,[],"[(""RI"", 785.664)]"
4,NY,2020-04-04,0.0,0.0,9773.5,21108.9,21804.5,695.613,1.03295,[],[]
5,NY,2020-04-05,0.0,0.0,9773.5,21108.9,23066.7,1957.83,1.09275,[],[]
6,NY,2020-04-06,0.0,0.0,9773.5,21108.9,24089.6,2980.67,1.1412,[],[]
7,NY,2020-04-07,0.0,0.0,9773.5,21108.9,24835.1,3726.23,1.17652,[],[]
8,NY,2020-04-08,0.0,0.0,9773.5,21108.9,25303.3,4194.35,1.1987,[],[]
9,NY,2020-04-09,1104.7,0.0,9773.5,20004.2,25484.6,5480.36,1.27396,"[(""DE"", 54.6573), (""PA"", 1050.04)]",[]


In [18]:
results.sent_to

Dict{String, Vector{String}} with 11 entries:
  "RI" => ["NY"]
  "NY" => ["CT", "DE", "MA", "MD", "NJ", "PA", "RI"]
  "ME" => ["NH", "RI"]
  "NJ" => []
  "DE" => ["NJ"]
  "MD" => ["NY"]
  "NH" => ["CT", "MA"]
  "CT" => ["NY"]
  "MA" => ["NY"]
  "PA" => ["NJ", "NY"]
  "VT" => ["MA", "NH"]